# Back to Julia

In [1]:
%load_ext Cython

This is what we had last time to start with...

## Pure python

In [4]:
from collections import namedtuple
Box = namedtuple("Box", "x1 x2 y1 y2")
bounds = Box(-1.8, 1.8, -1.8, 1.8)
focus=complex(-0.62772, -.42193)
gridsize=1000
iters=300

def setup_grid(gridsize, box):
    xstep = (box.x2 - box.x1)/(gridsize - 1.0)
    ystep = (box.y2 - box.y1)/(gridsize - 1.0)
    xs = (box.x1+ i* xstep for i in range(gridsize))
    zs=[]
    for x in xs:
        ys = (box.y1+ i* ystep for i in range(gridsize))
        for y in ys:
            zs.append(complex(x,y))
    return zs

def zts1(maxiter, zs, c): 
    output = [0] * len(zs)
    for i,z in enumerate(zs):
        n=0
        while n < maxiter and abs(z) < 2:
            z=z*z+c
            n+=1 
        output[i] = n
    return output

def run1():
    zs = setup_grid(gridsize, bounds)
    out = zts1(iters, zs, focus)
    return zs, out

In [5]:
%timeit -r 1 -n 5 run1()

And by the end of last lab you probably got to something like this...lets call it **B**

## cython

In [6]:
%%cython --annotate
cimport cython
@cython.boundscheck(False)
def zts1_cython4(int maxiter, zs, double complex c): 
    cdef unsigned int i, n
    cdef double complex z
    output = [0] * len(zs)
    i = 0
    for z in zs:
        n=0
        while n < maxiter and z.real*z.real + z.imag*z.imag < 4:
            z=z*z+c
            n+=1 
        output[i] = n
        i += 1
    return output

In [7]:
def run4():
    zs = setup_grid(gridsize, bounds)
    out = zts1_cython4(iters, zs, focus)
    return zs, out

In [8]:
%timeit -r 1 -n 5 run4()

## Q1. 

Implement the zts kernel such that `zs` is treated as a memoryview coming in and output is `cdef`ed as a memoryview to a `zs` shaped empty numpy array.

In [24]:
%%cython --annotate
cimport cython
import numpy as np
cimport numpy as np
@cython.boundscheck(False)
#your code here


Now implement the grid zs as a numpy zeros array. Keep our generator based complex number creation going for now...

In [ ]:
def setup_grid_numpy(gridsize, box):
    #your code here


In [30]:
def run5():
    zs = setup_grid_numpy1(gridsize, bounds)
    out = zts1_numpy(iters, zs, focus)
    return zs, out

In [31]:
%timeit -r 1 -n 5 run5()

## using `np.meshgrid`

In [32]:
def setup_grid_numpy2(gridsize, box):
    xs = np.linspace(box.x1, box.x2, gridsize)
    ys = np.linspace(box.y1, box.y2, gridsize)
    X,Y = np.meshgrid(xs,ys)
    zs = X + 1j*Y
    return zs.reshape(gridsize*gridsize,)

def run6():
    zs = setup_grid_numpy2(gridsize, bounds)
    out = zts1_numpy(iters, zs, focus)
    return zs, out

In [34]:
%timeit -r 1 -n 5 run6()

## Q2.

You'll notice that this intermediate `zs` array we created isnt actually needed. We could set up the `z`s, and immediately run the code which checks to see if the value of `z` escapes.  We have refactored some of this code for you below. Start with `setup_grid_numpy` and change it to create a `run_cython_amalgamated` function which computes the z and runs `escape` on it. This function should return a tuple of memoryviers `zs, output`.

The main loop should look like:
```python
for i in range(gridsize):
    x = xstart + i*xstep
    for j in range(gridsize):
        y = ystart + i*ystep
...
```

In [42]:
%%cython --annotate
cimport cython
import numpy as np
cimport numpy as np

cdef inline double norm2(double complex z):
    return z.real*z.real + z.imag*z.imag

cdef int escape(int maxiter, double complex z, double complex c):
    cdef int n=0
    while n < maxiter and norm2(z) < 4:
        z=z*z+c
        n+=1
    return n
    
@cython.boundscheck(False)
@cython.wraparound(False)
def run_cython_amalgamated(int gridsize, box, double complex c, int maxiter):
    #your code here


In [71]:
%timeit -r 1 -n 5 run_cython_amalgamated(gridsize, bounds, focus, iters)

## Q3.

Lets nor parallelize this function. Cython allows us to do this using `openmp`, which is supported by the gcc, not the clang compiler. 

First notice that we have added `nogil` to the `cdef` functions that are called from main `def` function. Since those are in C, they can give up the GIL. So this is the unusual situation where coding in C allows us to get parallelism just using threads.

The main loop should look like:
```python
for i in prange(gridsize, nogil=True,
                    schedule='static', chunksize=1):
    x = xstart + i*xstep
    for j in range(gridsize):
        y = ystart + i*ystep
...
```

What's happening here is that `prange` partitions the outside loop to run on multiple threads in `chunksize` blocks, with a `static` schedule fixed at compile time. 

First write all the code in a `%%cython` cell below with annotations to see what it looks like.

In [62]:
%%cython --annotate
cimport cython
import numpy as np
cimport numpy as np
from cython.parallel cimport prange

cdef inline double norm2(double complex z) nogil:
    return z.real*z.real + z.imag*z.imag

cdef int escape(int maxiter, double complex z, double complex c) nogil:
    cdef int n=0
    while n < maxiter and norm2(z) < 4:
        z=z*z+c
        n+=1
    return n

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef run_cython_amalgamated2(int gridsize, box, double complex c, int maxiter):
    #your code here


Then copy that all in a `parjulia.pyx` file with `openmp` compilation directives. To compile you will need gcc (`brew install gcc`), or a recent version of `clang`. Mine is 3.5 and openmp is not supported.

In [66]:
%%file parjulia.pyx
# distutils: extra_compile_args = -fopenmp
# distutils: extra_link_args = -fopenmp
#your code here


Here is a `setup.py` to compile it

In [67]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize
import numpy

setup(name="parjulia",
      ext_modules=cythonize("parjulia.pyx"),
     include_dirs=[numpy.get_include()])

And this is what I did to build the module:

In [75]:
!export CC="/usr/local/bin/gcc-5"; python setup.py build_ext -if

Lets run it!

In [69]:
from parjulia import run_cython_amalgamated2

In [72]:
%timeit -r 1 -n 5 run_cython_amalgamated2(gridsize, bounds, focus, iters)